# California housing prices 
.
Versão com o PySpark do kernel [California housing prices](https://github.com/robson-rsp/datascience/blob/main/regression/california_housing_prices.ipynb). Este notebook foi escrito no ambiente do Databricks e exportado como arquivo IPython .ipynb. Os textos daqui são cópias dos originais do link.
---
A tarefa neste kernel é criar um modelo de Machine Learning que estime preços de imóveis no estado da Califórnia baseado em características, como:

*   latitude/longitude: Localização do imóvel
*   housing_median_age: Idade média dos imóveis naquela quadra
*   totalRooms: Número total de cômodos dentro de um quarteirão
*   totalBedrooms: Número total de quartos dentro de um quarteirão
*   population: Número total de pessoas que moram em uma quadra
*   households: Número total de habitações em uma quadra
*   medianIncome: Renda média das famílias de uma quadra (medida em dezenas de milhares de dólares americanos)
*   medianHouseValue: Valor médio dos imóveis de uma quadra (medido em dólares americanos)
*   oceanProximity: Localização do imóvel em relação ao oceano/mar

Obs. A média populacional desses quarteirões é de 600 até 3000 pessoas.

Fonte: [Kaggle](https://www.kaggle.com/datasets/camnugent/california-housing-prices)

In [0]:

housing = spark.read.format("csv") \
          .option("inferSchema", "True") \
          .option("header", "True") \
          .option("sep", ",") \
          .load("/FileStore/tables/housing.csv")

# EDA

In [0]:
housing.sample(fraction=0.01).toPandas()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.28,37.82,52.0,1238.0,288.0,622.0,259.0,1.5057,87500.0,NEAR BAY
1,-122.28,37.81,52.0,386.0,164.0,346.0,155.0,0.8075,137500.0,NEAR BAY
2,-122.28,37.90,52.0,2261.0,328.0,819.0,335.0,4.9083,346800.0,NEAR BAY
3,-122.28,37.89,52.0,2315.0,408.0,835.0,369.0,4.5893,290100.0,NEAR BAY
4,-122.29,37.88,52.0,1650.0,395.0,841.0,380.0,3.5560,179300.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
201,-122.81,40.93,16.0,2050.0,471.0,588.0,195.0,2.7083,88900.0,INLAND
202,-119.27,36.34,26.0,2057.0,472.0,1453.0,439.0,2.4113,58600.0,INLAND
203,-119.21,34.19,15.0,3797.0,692.0,2216.0,675.0,4.7443,229500.0,NEAR OCEAN
204,-118.93,34.20,17.0,2619.0,606.0,1655.0,557.0,3.8860,281300.0,<1H OCEAN


In [0]:
print(f'shape={housing.count(), len(housing.columns)}')

shape=(20640, 10)


In [0]:
housing.summary("count", "mean", "stddev", "min", "25%", "50%", "75%", "max").toPandas()

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20433,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,537.8705525375618,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,421.38507007403115,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,25%,-121.8,33.93,18.0,1447.0,296.0,787.0,280.0,2.5625,119600.0,None
5,50%,-118.49,34.26,29.0,2127.0,435.0,1166.0,409.0,3.5347,179700.0,None
6,75%,-118.01,37.71,37.0,3146.0,647.0,1724.0,605.0,4.7426,264700.0,None
7,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


Verificando a ocorrência de campos NaN.

In [0]:
for name in housing.columns:
    mask = housing[name].isNull()
    print(f'{name}: {housing.where(mask).count()}')

longitude: 0
latitude: 0
housing_median_age: 0
total_rooms: 0
total_bedrooms: 207
population: 0
households: 0
median_income: 0
median_house_value: 0
ocean_proximity: 0


# Data visualization

Os gráficos foram criados utilizando as ferramentas de visualização nativas do Databricks. O resultado não pode ser visto em um arquivo .ipynb

# Train/Test split

In [0]:
train_set, test_set = housing.randomSplit([0.8, 0.2])

# Feature engineering

In [0]:
from pyspark.ml.feature import Imputer, StandardScaler, StringIndexer, VectorAssembler

## Categorical features

**Etapa ##:** Atributos nominais: Codificar

In [0]:
si = StringIndexer(inputCol='ocean_proximity', outputCol='ocean_proximity_')
si_model = si.fit(train_set)
train_set = si_model.transform(train_set)
test_set  = si_model.transform(test_set)

## Numerical features

**Etapa ##:** NaN values: Imputar

In [0]:
im = Imputer(strategy='mean', inputCol='total_bedrooms', outputCol='total_bedrooms_')
im_model = im.fit(train_set)
train_set = im_model.transform(train_set)
test_set  = im_model.transform(test_set)

**Etapa ##:** Os atributos 'total_rooms' e 'total_bedrooms' não fazem sentido se eu deixá-los da forma como estão. 'total_rooms' refere-se à quantidade total de cômodos que há em cada quadra. 'total_bedrooms' refere-se à quantidade total de quartos que há em cada quadra. Vou criar novos atributos a partir dos existentes. Vou fazer isso dentro de uma classe para usar a classe Pipeline.
1. **mean_bedrooms_households:** Quantidade média de quartos por habitação.
1. **mean_rooms_households:** Quantidade média de cômodos por habitação.
1. **mean_population_households:** Quantidade média de moradores por imóvel.

In [0]:
train_set = train_set.withColumn('mean_bedrooms_households', train_set['total_bedrooms_'] / train_set['households'])
train_set = train_set.withColumn('mean_rooms_households', train_set['total_rooms'] / train_set['households'])
train_set = train_set.withColumn('mean_population_households', train_set['population'] / train_set['households'])

test_set = test_set.withColumn('mean_bedrooms_households', test_set['total_bedrooms_'] / test_set['households'])
test_set = test_set.withColumn('mean_rooms_households', test_set['total_rooms'] / test_set['households'])
test_set = test_set.withColumn('mean_population_households', test_set['population'] / test_set['households'])

**Etapa ##:** Unir todas as colunas em uma única que será utilizada pelos modelos de Machine Learning do PySpark. Essa etapa dever ser feita antes da padronização dos atributos com StandardScaler.

In [0]:
cols = ['longitude', 'latitude', 'housing_median_age', 'population', 'households', 'median_income', 'median_house_value', 'ocean_proximity_', 'total_bedrooms_', 'mean_bedrooms_households', 'mean_rooms_households', 'mean_population_households']

va = VectorAssembler(inputCols=cols, outputCol='X')
train_set = va.transform(train_set)
test_set  = va.transform(test_set)

**Etapa ##:** Padronização dos atributos com StandardScaler.

In [0]:
ss = StandardScaler(inputCol='X', outputCol='X_')
ss_model = ss.fit(train_set)
train_set = ss_model.transform(train_set)
test_set  = ss_model.transform(test_set)

**Etapa ##:** Selecionar X e y dos conjuntos de treino e teste. Vou renomear "X_" para "X"  e "median_house_value" para "y" assim os conjuntos ficarão de acordo com o padrão de nomenclatura.

In [0]:
train_set = train_set.select('X_', 'median_house_value')
test_set  = test_set.select('X_', 'median_house_value')

train_set = train_set.withColumnRenamed(existing='X_', new='X')
test_set  = test_set.withColumnRenamed(existing='X_', new='X')

train_set = train_set.withColumnRenamed(existing='median_house_value', new='y')
test_set  = test_set.withColumnRenamed(existing='median_house_value', new='y')

# Model training


In [0]:
from pyspark.ml.regression import LinearRegression, RandomForestRegressor

In [0]:
lr = LinearRegression(featuresCol='X', labelCol='y', elasticNetParam=0.0)
lr = lr.fit(train_set)
y_pred = lr.evaluate(test_set)
y_pred.r2, y_pred.meanAbsoluteError
print(f'r2:  {y_pred.r2}')
print(f'mae: {y_pred.meanAbsoluteError}')
print(f'mse: {y_pred.meanSquaredError}')
y_pred.predictions.show()

r2:  1.0
mae: 3.036743201726002e-07
mse: 1.682191870773665e-13
+--------------------+--------+------------------+
|                   X|       y|        prediction|
+--------------------+--------+------------------+
|[-61.961585568029...|106700.0|106699.99999962175|
|[-61.951610266481...| 68400.0| 68400.00000012563|
|[-61.936647314158...| 70500.0| 70499.99999969549|
|[-61.936647314158...| 64600.0|64599.999999710715|
|[-61.931659663384...| 86400.0| 86399.99999956966|
|[-61.931659663384...|105900.0|105900.00000025496|
|[-61.926672012610...| 85100.0| 85099.99999967922|
|[-61.926672012610...| 85600.0| 85599.99999973827|
|[-61.926672012610...| 81800.0| 81799.99999973776|
|[-61.926672012610...| 74700.0|  74699.9999998099|
|[-61.921684361835...| 86400.0| 86399.99999974194|
|[-61.921684361835...|103100.0|103100.00000040485|
|[-61.916696711061...| 75100.0| 75099.99999934474|
|[-61.901733758739...| 96100.0| 96099.99999971352|
|[-61.896746107964...| 55000.0| 54999.99999945175|
|[-61.896746107964.

In [0]:
rf = RandomForestRegressor(featuresCol='X', labelCol='y')
rf = rf.fit(train_set)